<a href="https://colab.research.google.com/github/Parkseoho/DAStudy-sun2/blob/main/SEOHO/%EA%B3%B5%EA%B3%B5%EA%B8%B0%EA%B4%80_%EA%B8%B0%EC%83%81_seoho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 서울시 공공자전거 일별 대여건수 22년 1년치 데이터 불러오기

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

bike = pd.read_csv('/content/서울특별시 공공자전거 일별 대여건수_22.01-22.06.csv',encoding='cp949')

## 서울시 22년도 1년치 기상데이터 api연동 그리고 보기 편한형태로 변경

In [6]:
import json

url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
params ={'serviceKey' : "api키", 'pageNo' : '1', 'numOfRows' : '365', 'dataType' : 'JSON', 'dataCd' : 'ASOS', 'dateCd' : 'DAY', 'startDt' : '20220101', 'endDt' : '20221231', 'stnIds' : '108' }

response = requests.get(url, params=params)
data = json.loads(response.text)
df = pd.json_normalize(data)
df2 = df.explode('response.body.items.item')
df2 = pd.json_normalize(df2['response.body.items.item'])



In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 62 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   stnId           365 non-null    object
 1   stnNm           365 non-null    object
 2   tm              365 non-null    object
 3   avgTa           365 non-null    object
 4   minTa           365 non-null    object
 5   minTaHrmt       365 non-null    object
 6   maxTa           365 non-null    object
 7   maxTaHrmt       365 non-null    object
 8   mi10MaxRn       365 non-null    object
 9   mi10MaxRnHrmt   365 non-null    object
 10  hr1MaxRn        365 non-null    object
 11  hr1MaxRnHrmt    365 non-null    object
 12  sumRnDur        365 non-null    object
 13  sumRn           365 non-null    object
 14  maxInsWs        365 non-null    object
 15  maxInsWsWd      365 non-null    object
 16  maxInsWsHrmt    365 non-null    object
 17  maxWs           365 non-null    object
 18  maxWsWd   

In [8]:
df2.describe()

,stnId,stnNm,tm,avgTa,minTa,minTaHrmt,maxTa,maxTaHrmt,mi10MaxRn,mi10MaxRnHrmt,...,avgM05Te,avgM10Te,avgM15Te,avgM30Te,avgM50Te,sumLrgEv,sumSmlEv,n99Rn,iscs,sumFogDur
count,365,365,365,365,365,365,365,365,365,365,...,365,365,365,365,365,365,365,365,365,365
unique,1,1,365,223,235,222,227,200,43,61,...,186,190,188,128,59,66,90,70,209,2
top,108,서울,2022-01-01,26.8,22.0,0001,28.4,0001,,,...,1.2,26.3,23.9,10.2,19.0,1.5,1.8,,,
freq,365,365,1,6,5,8,5,21,271,304,...,26,12,10,14,32,18,16,217,155,364


In [65]:
# 각 칼럼의 unique한 값 수치 확인 후 1인것은 상관관계 연관없으니 제거

for i in df2.columns:
  print(i, df2[i].nunique())

tm 365
avgTa 223
minTa 235
minTaHrmt 222
maxTa 227
maxTaHrmt 200
mi10MaxRn 43
mi10MaxRnHrmt 61
hr1MaxRn 57
hr1MaxRnHrmt 61
sumRnDur 94
sumRn 73
maxInsWs 103
maxInsWsWd 15
maxInsWsHrmt 297
maxWs 56
maxWsWd 15
maxWsHrmt 293
avgWs 35
hr24SumRws 333
maxWd 13
avgTd 248
minRhm 71
minRhmHrmt 248
avgRhm 242
avgPv 201
avgPa 212
maxPs 200
maxPsHrmt 224
minPs 213
minPsHrmt 248
avgPs 220
ssDur 53
sumSsHr 116
hr1MaxIcsrHrmt 10
hr1MaxIcsr 219
sumGsr 343
ddMefs 13
ddMefsHrmt 16
ddMes 23
ddMesHrmt 17
sumDpthFhsc 13
avgTca 80
avgLmac 72
avgTs 229
minTg 238
avgCm5Te 200
avgCm10Te 200
avgCm20Te 189
avgCm30Te 177
avgM05Te 186
avgM10Te 190
avgM15Te 188
avgM30Te 128
avgM50Te 59
sumLrgEv 66
sumSmlEv 90
n99Rn 70
iscs 209


In [18]:
#  의미 없는 칼럼 제거
df2 = df2.drop(columns=['stnId','stnNm',"sumFogDur"])

### 자전거 대여 데이터와 기상데이터간 결합
### 그리고 간단한 상관관계 확인을 위해 데이터의 type을 float으로 바꾸고 공백인 값을 0으로 변경

In [22]:
df_com =  df2.merge(bike,left_on='tm',right_on='대여일시')
df_com['대여건수'] = df_com['대여건수'].str.replace(',','')
df_com['대여건수'] = df_com['대여건수'].astype(int)
df_com=df_com.replace(' ',0)

for i in df_com.drop(columns=['tm','iscs','대여일시','대여건수']).columns:
  df_com[i] = df_com[i].astype(float)

In [69]:
# tm, iscs, 대여일시의 경우 숫자타입이아닌 문자타입이기때문에 제외
# 대여건수를 기준으로 각 칼럼의 상관관계 오름차순 정렬

df_com.drop(columns=['tm','iscs','대여일시']).corr()['대여건수'].sort_values(ascending=False)

대여건수              1.000000
maxTa             0.728217
avgTs             0.712753
sumLrgEv          0.681742
avgCm5Te          0.681343
sumSmlEv          0.680189
avgCm10Te         0.677810
avgTa             0.676644
avgCm20Te         0.676550
avgCm30Te         0.672461
avgM05Te          0.662671
minTa             0.634863
avgM10Te          0.627606
sumGsr            0.580140
avgTd             0.571920
avgM15Te          0.568900
hr1MaxIcsr        0.565079
minTg             0.546169
ssDur             0.524498
avgPv             0.402343
avgM30Te          0.299913
minRhmHrmt        0.281469
sumSsHr           0.280123
maxTaHrmt         0.231354
hr1MaxIcsrHrmt    0.039562
avgRhm            0.032039
maxPsHrmt         0.022895
maxWsHrmt        -0.022149
minPsHrmt        -0.047497
avgTca           -0.050530
maxInsWsHrmt     -0.053047
maxInsWsWd       -0.062612
maxWsWd          -0.073137
minRhm           -0.073348
avgM50Te         -0.074760
avgLmac          -0.120937
hr1MaxRnHrmt     -0.125595
m

In [67]:
# 음의 상관관계 높은 순위
# 1. maxPs 최고 해면 기압
# 2. avgPs 평균 해면기압


# 양의 상관관계 높은 순위
# 1. maxTa : 최고 기온
# 2. avgTs : 평균 지면온도
# 3. sumLrgEv : 합계 대형증발량
# 4. avgCm5Te : 평균 5cm 지중온도
# 5. sumSmlEv : 합계 소형증발
# 6. avgCm10Te : 평균 5cm 지중온도
# 7. avgTa : 평균 기온


# - 다음주 해야할 분석
# 음의 상관관계 상위와 양의 상관관계 상위 컬럼을 기반으로 대여건수와 상관관계 분석